# 피봇테이블과 그룹분석

## 피봇

In [1]:
import pandas as pd

In [2]:
data = {
    "도시": ["서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"],
    "연도": ["2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"],
    "인구": [9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
    "지역": ["수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"]
}
columns = ["도시", "연도", "인구", "지역"]
df1 = pd.DataFrame(data, columns=columns)
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [3]:
df2 = df1[["도시", "연도", "인구"]]
df2

,도시,연도,인구
0,서울,2015,9904312
1,서울,2010,9631482
2,서울,2005,9762546
3,부산,2015,3448737
4,부산,2010,3393191
5,부산,2005,3512547
6,인천,2015,2890451
7,인천,2010,263203


In [4]:
df2.pivot(index="도시", columns="연도", values="인구")

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


In [5]:
df2.pivot(index="연도", columns="도시", values="인구")

도시,부산,서울,인천
연도,,,
2005,3512547.0,9762546.0,NaN
2010,3393191.0,9631482.0,263203.0
2015,3448737.0,9904312.0,2890451.0


In [8]:
# 피벗없이 만들기
df2.set_index(['도시', '연도']).unstack()

인구                      
연도       2005       2010       2015
도시                                 
부산  3512547.0  3393191.0  3448737.0
서울  9762546.0  9631482.0  9904312.0
인천        NaN   263203.0  2890451.0

행 인덱스나 열 인덱스를 리스트로 주는 경우에는 다중 인덱스 피봇 테이블을 생성한다. (주의: 판다스 버전 1.1 미만에서는 버그로 인해 동작하지 않는다.)


In [9]:
df1.pivot(index= ["지역", "도시"], columns="연도", values="인구")

연도           2005       2010       2015
지역  도시                                 
경상권 부산  3512547.0  3393191.0  3448737.0
수도권 서울  9762546.0  9631482.0  9904312.0
    인천        NaN   263203.0  2890451.0

In [10]:
try:
    df1.pivot(index="지역", columns="연도", values="인구")
except ValueError as e:
    print("ValueError:", e)

ValueError: Index contains duplicate entries, cannot reshape


## 그룹분석
1. 분석하고자 하는 시리즈나 데이터프레임에 groupby 메서드를 호출하여 그룹화를 한다.
2. 그룹 객체에 대해 그룹연산을 수행한다.

### groupby 메서드

### 그룹연산 메서드

In [12]:
import numpy as np

np.random.seed(0)
df2 = pd.DataFrame({
    'key1': ['A', 'A', 'B', 'B', 'A'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': [1, 2, 3, 4, 5],
    'data2': [10, 20, 30, 40, 50]
})
df2

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


In [14]:
df2.groupby?

Signature:
df2.groupby(
    by=None,
    axis: 'Axis | lib.NoDefault' = <no_default>,
    level: 'IndexLabel | None' = None,
    as_index: 'bool' = True,
    sort: 'bool' = True,
    group_keys: 'bool' = True,
    observed: 'bool | lib.NoDefault' = <no_default>,
    dropna: 'bool' = True,
) -> 'DataFrameGroupBy'
Docstring:
Group DataFrame using a mapper or by a Series of columns.

A groupby operation involves some combination of splitting the
object, applying a function, and combining the results. This can be
used to group large amounts of data and compute operations on these
groups.

Parameters
----------
by : mapping, function, label, pd.Grouper or list of such
    Used to determine the groups for the groupby.
    If ``by`` is a function, it's called on each value of the object's
    index. If a dict or Series is passed, the Series or dict VALUES
    will be used to determine the groups (the Series' values are first
    aligned; see ``.align()`` method). If a list or ndarray of lengt

In [13]:
groups = df2.groupby(df2.key1)
groups

In [15]:
groups.groups

{'A': [0, 1, 4], 'B': [2, 3]}

In [16]:
groups.sum()

,key2,data1,data2
key1,,,
A,onetwoone,8,80
B,onetwo,7,70


In [17]:
df2

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


In [18]:
df2.data1.groupby(df2.key1).sum()

key1
A    8
B    7
Name: data1, dtype: int64

In [19]:
df2.groupby(df2.key1)["data1"].sum()   # `GroupBy` 클래스 객체에서 data1만 선택하여 분석하는 경우

key1
A    8
B    7
Name: data1, dtype: int64

In [20]:
df2.groupby(df2.key1).sum()["data1"]  # 전체 데이터를 분석한 후 data1만 선택한 경우

key1
A    8
B    7
Name: data1, dtype: int64

### 연습문제. groupby
key1의 값을 기준으로 data1의 값을 분류하여 합계를 구한 결과를 시리즈가 아닌 데이터프레임으로 구한다.

In [21]:
df2.groupby(df2.key1)[["data1"]].sum()

,data1
key1,
A,8
B,7


In [22]:
df2

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


이번에는 복합 키 (key1, key2) 값에 따른 data1의 합계를 구하자.

In [24]:
df2.data1.groupby([df2.key1, df2.key2]).sum()

key1  key2
A     one     6
      two     2
B     one     3
      two     4
Name: data1, dtype: int64

In [25]:
df2.data1.groupby([df2["key1"], df2["key2"]]).sum().unstack("key2")

key2,one,two
key1,,
A,6,2
B,3,4


In [26]:
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


그룹분석 기능을 사용하면 위의 인구 데이터로부터 지역별 합계를 구할 수도 있다.

In [27]:
df1["인구"].groupby(df1["지역"]).sum()

지역
경상권    10354475
수도권    32451994
Name: 인구, dtype: int64

In [32]:
df1["인구"].groupby([df1["지역"], df1["연도"]]).sum().unstack("연도")

연도,2005,2010,2015
지역,,,
경상권,3512547,3393191,3448737
수도권,9762546,9894685,12794763


다음 데이터는 150 송이의 붓꽃(iris)에 대해 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width), 꽃잎폭(sepal_width), 꽃잎폭(sepal_width)을 측정한 데이터이다.

In [36]:
import seaborn as sns
iris = sns.load_dataset("iris")
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [37]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [38]:
def peak_to_peak_ratio(x):
    return x.max() / x.min()

iris.groupby(iris.species).agg(peak_to_peak_ratio)

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,1.348837,1.913043,1.900000,6.000000
versicolor,1.428571,1.700000,1.700000,1.800000
virginica,1.612245,1.727273,1.533333,1.785714


In [40]:
iris.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [43]:
iris.groupby(iris.species).describe()

sepal_length                                              \
                  count   mean       std  min    25%  50%  75%  max   
species                                                               
setosa             50.0  5.006  0.352490  4.3  4.800  5.0  5.2  5.8   
versicolor         50.0  5.936  0.516171  4.9  5.600  5.9  6.3  7.0   
virginica          50.0  6.588  0.635880  4.9  6.225  6.5  6.9  7.9   

           sepal_width         ... petal_length      petal_width         \
                 count   mean  ...          75%  max       count   mean   
species                        ...                                        
setosa            50.0  3.428  ...        1.575  1.9        50.0  0.246   
versicolor        50.0  2.770  ...        4.600  5.1        50.0  1.326   
virginica         50.0  2.974  ...        5.875  6.9        50.0  2.026   

                                               
                 std  min  25%  50%  75%  max  
species                                        
setosa      0.105386  0.1  0.2  0.2  0.3  0.6  
versicolor  0.197753  1.0  1.2  1.3  1.5  1.8  
virginica   0.274650  1.4  1.8  2.0  2.3  2.5  

[3 rows x 32 columns]

In [44]:
iris.groupby(iris.species).describe().T

species                setosa  versicolor  virginica
sepal_length count  50.000000   50.000000  50.000000
             mean    5.006000    5.936000   6.588000
             std     0.352490    0.516171   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
sepal_width  count  50.000000   50.000000  50.000000
             mean    3.428000    2.770000   2.974000
             std     0.379064    0.313798   0.322497
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.525000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.675000    3.000000   3.175000
             max     4.400000    3.400000   3.800000
petal_length count  50.000000   50.000000  50.000000
             mean    1.462000    4.260000   5.552000
             std     0.173664    0.469911   0.551895
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.350000   5.550000
             75%     1.575000    4.600000   5.875000
             max     1.900000    5.100000   6.900000
petal_width  count  50.000000   50.000000  50.000000
             mean    0.246000    1.326000   2.026000
             std     0.105386    0.197753   0.274650
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000

In [45]:
iris.sort_values(by="petal_length", ascending=False)[:3]

,sepal_length,sepal_width,petal_length,petal_width,species
118,7.7,2.6,6.9,2.3,virginica
122,7.7,2.8,6.7,2.0,virginica
117,7.7,3.8,6.7,2.2,virginica


In [46]:
def top3_petal_length(df):
    return df.sort_values(by="petal_length", ascending=False)[:3]

iris.groupby(iris.species).apply(top3_petal_length)

sepal_length  sepal_width  petal_length  petal_width  \
species                                                                
setosa     24            4.8          3.4           1.9          0.2   
           44            5.1          3.8           1.9          0.4   
           23            5.1          3.3           1.7          0.5   
versicolor 83            6.0          2.7           5.1          1.6   
           77            6.7          3.0           5.0          1.7   
           72            6.3          2.5           4.9          1.5   
virginica  118           7.7          2.6           6.9          2.3   
           117           7.7          3.8           6.7          2.2   
           122           7.7          2.8           6.7          2.0   

                   species  
species                     
setosa     24       setosa  
           44       setosa  
           23       setosa  
versicolor 83   versicolor  
           77   versicolor  
           72   versicolor  
virginica  118   virginica  
           117   virginica  
           122   virginica

In [47]:
def q3cut(s):
    return pd.qcut(s, 3, labels=["소", "중", "대"]).astype(str)


iris["petal_length_class"] = iris.groupby(iris.species).petal_length.transform(q3cut)
iris[["petal_length", "petal_length_class"]].tail(10)

,petal_length,petal_length_class
140,5.6,중
141,5.1,소
142,5.1,소
143,5.9,대
144,5.7,중
145,5.2,소
146,5.0,소
147,5.2,소
148,5.4,중
149,5.1,소


### 연습문제. iris 종별 평균
붓꽃(iris) 데이터에서 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width) 등의 평균을 구하라.   
만약 붓꽃 종(species)이 표시되지 않았을 때 이 수치들을 이용하여 붓꽃 종을 찾아낼 수 있을지 생각하라.

In [50]:
import seaborn as sns
iris = sns.load_dataset("iris")
iris.groupby('species').mean()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


## 피봇 테이블
Pandas는 pivot 명령과 groupby 명령의 중간 성격을 가지는 pivot_table 명령도 제공한다.

- pivot_table 명령은 groupby 명령처럼 그룹분석을 하지만 최종적으로는 pivot 명령처럼 피봇테이블을 만든다.
- 즉 groupby 명령의 결과에 unstack을 자동 적용하여 2차원적인 형태로 변형한다. 사용 방법은 다음과 같다.

pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, margins_name='All')
- data: 분석할 데이터프레임 (메서드일 때는 필요하지 않음)
- values: 분석할 데이터프레임에서 분석할 열
- index: 행 인덱스로 들어갈 키 열 또는 키 열의 리스트
- columns: 열 인덱스로 들어갈 키 열 또는 키 열의 리스트
- aggfunc: 분석 메서드
- fill_value: NaN 대체 값
- margins: 모든 데이터를 분석한 결과를 오른쪽과 아래에 붙일지 여부
- margins_name: 마진 열(행)의 이름


만약 조건에 따른 데이터가 유일하게 선택되지 않으면 그룹연산을 하며 이 때 aggfunc 인수로 정의된 함수를 수행하여 대표값을 계산한다.
pivot_table를 메서드로 사용할 때는 객체 자체가 데이터가 되므로 data 인수가 필요하지 않다.
예를 들어 위에서 만들었던 피봇테이블은 pivot_table 명령으로 다음과 같이 만들 수도 있다. 인수의 순서에 주의하라.

In [51]:
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [52]:
df1.pivot_table(values="인구", index="도시", columns="연도") # 인자를 안써도 되지만, 헷갈리니 꼭 기입해주자

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


In [53]:
df1.pivot_table(values="인구", index="도시", columns="연도", margins=True, margins_name="평균") # aggfunc 디폴트 평균계산

연도,2005,2010,2015,평균
도시,,,,
부산,3512547.0,3393191.0,3448737.0,3.451492e+06
서울,9762546.0,9631482.0,9904312.0,9.766113e+06
인천,NaN,263203.0,2890451.0,1.576827e+06
평균,6637546.5,4429292.0,5414500.0,5.350809e+06


In [54]:
df1["인구"].mean()

5350808.625

In [56]:
df1.pivot_table("인구", index=["연도", "도시"])

인구
연도   도시           
2005 부산  3512547.0
     서울  9762546.0
2010 부산  3393191.0
     서울  9631482.0
     인천   263203.0
2015 부산  3448737.0
     서울  9904312.0
     인천  2890451.0

In [4]:
import seaborn as sns
tips = sns.load_dataset("tips")
tips.tail()

,total_bill,tip,sex,smoker,day,time,size
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2
243,18.78,3.00,Female,No,Thur,Dinner,2


In [5]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.tail()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204
243,18.78,3.00,Female,No,Thur,Dinner,2,0.159744


In [6]:
tips.describe()

,total_bill,tip,size,tip_pct
count,244.000000,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672,0.160803
std,8.902412,1.383638,0.951100,0.061072
min,3.070000,1.000000,1.000000,0.035638
25%,13.347500,2.000000,2.000000,0.129127
50%,17.795000,2.900000,2.000000,0.154770
75%,24.127500,3.562500,3.000000,0.191475
max,50.810000,10.000000,6.000000,0.710345


In [7]:
tips.groupby("sex").count()

/var/folders/gb/jqxp5nqd3dz14l_g216xwzdm0000gn/T/ipykernel_10834/294839457.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tips.groupby("sex").count()


,total_bill,tip,smoker,day,time,size,tip_pct
sex,,,,,,,
Male,157,157,157,157,157,157,157
Female,87,87,87,87,87,87,87


In [8]:
tips.groupby("sex").size()

/var/folders/gb/jqxp5nqd3dz14l_g216xwzdm0000gn/T/ipykernel_10834/2787469288.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tips.groupby("sex").size()


sex
Male      157
Female     87
dtype: int64

In [9]:
tips.groupby(["sex", "smoker"]).count()

/var/folders/gb/jqxp5nqd3dz14l_g216xwzdm0000gn/T/ipykernel_10834/1168168671.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tips.groupby(["sex", "smoker"]).count()


total_bill  tip  day  time  size  tip_pct
sex    smoker                                           
Male   Yes             60   60   60    60    60       60
       No              97   97   97    97    97       97
Female Yes             33   33   33    33    33       33
       No              54   54   54    54    54       54

In [10]:
tips.groupby(["sex", "smoker"]).count()['total_bill']

/var/folders/gb/jqxp5nqd3dz14l_g216xwzdm0000gn/T/ipykernel_10834/39944383.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tips.groupby(["sex", "smoker"]).count()['total_bill']


sex     smoker
Male    Yes       60
        No        97
Female  Yes       33
        No        54
Name: total_bill, dtype: int64

In [11]:
tips.groupby(["sex", "smoker"]).size()

/var/folders/gb/jqxp5nqd3dz14l_g216xwzdm0000gn/T/ipykernel_10834/4151768403.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tips.groupby(["sex", "smoker"]).size()


sex     smoker
Male    Yes       60
        No        97
Female  Yes       33
        No        54
dtype: int64

In [12]:
tips.pivot_table(values="tip_pct", index="sex", columns="smoker", aggfunc="count", margins=True)

smoker,Yes,No,All
sex,,,
Male,60,97,157
Female,33,54,87
All,93,151,244


In [13]:
# 성별과 흡연 여부에 따른 평균 팁 비율
tips.pivot_table("tip_pct", "sex", "smoker") # data, index, columns

smoker,Yes,No
sex,,
Male,0.152771,0.160669
Female,0.182150,0.156921


In [14]:
tips.groupby("sex")[["tip_pct"]].describe()

/var/folders/gb/jqxp5nqd3dz14l_g216xwzdm0000gn/T/ipykernel_10834/929131627.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tips.groupby("sex")[["tip_pct"]].describe()


tip_pct                                                              \
         count      mean       std       min       25%       50%       75%   
sex                                                                          
Male     157.0  0.157651  0.064778  0.035638  0.121389  0.153492  0.186240   
Female    87.0  0.166491  0.053632  0.056433  0.140416  0.155581  0.194266   

                  
             max  
sex               
Male    0.710345  
Female  0.416667

In [15]:
tips.groupby("smoker")[["tip_pct"]].describe()

/var/folders/gb/jqxp5nqd3dz14l_g216xwzdm0000gn/T/ipykernel_10834/499673867.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tips.groupby("smoker")[["tip_pct"]].describe()


tip_pct                                                              \
         count      mean       std       min       25%       50%       75%   
smoker                                                                       
Yes       93.0  0.163196  0.085119  0.035638  0.106771  0.153846  0.195059   
No       151.0  0.159328  0.039910  0.056797  0.136906  0.155625  0.185014   

                  
             max  
smoker            
Yes     0.710345  
No      0.291990

In [16]:
tips.groupby(["sex", "smoker"])[["tip_pct"]].describe()

/var/folders/gb/jqxp5nqd3dz14l_g216xwzdm0000gn/T/ipykernel_10834/3762351863.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tips.groupby(["sex", "smoker"])[["tip_pct"]].describe()


tip_pct                                                    \
                count      mean       std       min       25%       50%   
sex    smoker                                                             
Male   Yes       60.0  0.152771  0.090588  0.035638  0.101845  0.141015   
       No        97.0  0.160669  0.041849  0.071804  0.131810  0.157604   
Female Yes       33.0  0.182150  0.071595  0.056433  0.152439  0.173913   
       No        54.0  0.156921  0.036421  0.056797  0.139708  0.149691   

                                   
                    75%       max  
sex    smoker                      
Male   Yes     0.191697  0.710345  
       No      0.186220  0.291990  
Female Yes     0.198216  0.416667  
       No      0.181630  0.252672

### 연습문제. tips
1. 팁의 비율이 요일과 점심/저녁 여부, 인원수에 어떤 영향을 받는지 살펴본다.
2. 어떤 요인이 가장 크게 작용하는지 판단할 수 있는 방법이 있는가?

In [18]:
tips.groupby(['day','time', 'size'])[['tip']].mean()

/var/folders/gb/jqxp5nqd3dz14l_g216xwzdm0000gn/T/ipykernel_10834/3075668863.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tips.groupby(['day','time', 'size'])[['tip']].mean()


tip
day  time   size          
Thur Lunch  1     1.830000
            2     2.430638
            3     2.692500
            4     4.218000
            5     5.000000
            6     5.300000
     Dinner 1          NaN
            2     3.000000
            3          NaN
            4          NaN
            5          NaN
            6          NaN
Fri  Lunch  1     1.920000
            2     2.352000
            3     3.000000
            4          NaN
            5          NaN
            6          NaN
     Dinner 1          NaN
            2     2.777273
            3          NaN
            4     4.730000
            5          NaN
            6          NaN
Sat  Lunch  1          NaN
            2          NaN
            3          NaN
            4          NaN
            5          NaN
            6          NaN
     Dinner 1     1.000000
            2     2.517547
            3     3.797778
            4     4.123846
            5     3.000000
            6          NaN
Sun  Lunch  1          NaN
            2          NaN
            3          NaN
            4          NaN
            5          NaN
            6          NaN
     Dinner 1          NaN
            2     2.816923
            3     3.120667
            4     4.087778
            5     4.046667
            6     5.000000

In [75]:
tips

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204


In [78]:
tips.pivot_table(values='tip_pct', index=['day', 'time' ,'size'], columns='sex')

sex                   Male    Female
day  time   size                    
Thur Lunch  1          NaN  0.181728
            2     0.166265  0.161686
            3     0.154660  0.134538
            4     0.170948  0.128560
            5     0.121389       NaN
            6     0.195335  0.162891
     Dinner 2          NaN  0.159744
Fri  Lunch  1     0.223776       NaN
            2     0.149328  0.203729
            3          NaN  0.187735
     Dinner 2     0.132278  0.199115
            4     0.117750       NaN
Sat  Dinner 1          NaN  0.231832
            2     0.156128  0.153788
            3     0.150100  0.154919
            4     0.143348  0.110468
            5     0.106572       NaN
Sun  Dinner 2     0.173532  0.214411
            3     0.142529  0.167863
            4     0.154931  0.146998
            5     0.153661  0.172194
            6     0.103799       NaN